# 01 — Preprocessing (Real Transactions, Time‑Aware Split)
This notebook:
- Loads IEEE-CIS train_transaction + train_identity
- Cleans and engineers interpretable features
- Creates a chronological Train/Test split (real transactions only)
- Saves:
  - X_train_lgb.csv / X_test_lgb.csv (raw engineered features for LightGBM)
  - X_train_scaled.csv / X_test_scaled.csv (OHE + scaled features for unsupervised models)
  - y_train.csv / y_test.csv
  - train_keys.csv / test_keys.csv (row_id ↔ TransactionID/DT mapping)


In [1]:

# Install minimal requirements (Colab-safe)
!pip -q install -U scikit-learn==1.3.2 imbalanced-learn==0.12.3 pandas==2.2.2 numpy==1.26.4 joblib==1.4.2

import os, sys
import numpy as np
import pandas as pd
import joblib

# Reproducibility
SEED = 42
np.random.seed(SEED)

# Detect Colab + mount Drive
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')

# Adjust if needed
datapath = '/content/drive/MyDrive/RThesis/' if IN_COLAB else './'
os.makedirs(datapath, exist_ok=True)

print("datapath =", datapath)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
datapath = /content/drive/MyDrive/RThesis/


In [2]:

# --- Load + merge datasets (IEEE-CIS)
trans_path = os.path.join(datapath, "train_transaction (1).csv")
iden_path  = os.path.join(datapath, "train_identity (1).csv")

assert os.path.exists(trans_path), f"Missing file: {trans_path}"
assert os.path.exists(iden_path),  f"Missing file: {iden_path}"

trans = pd.read_csv(trans_path)
iden  = pd.read_csv(iden_path)

data = trans.merge(iden, how="left", on="TransactionID")
print("Merged shape:", data.shape)

# Basic sanity
required_cols = ["TransactionID", "TransactionDT", "isFraud"]
missing = [c for c in required_cols if c not in data.columns]
assert not missing, f"Missing required columns: {missing}"


Merged shape: (590540, 434)


In [3]:

# --- Basic cleaning: fill missing values
# Numeric -> median; Categorical -> "missing"
num_cols = data.select_dtypes(include=[np.number]).columns
data[num_cols] = data[num_cols].fillna(data[num_cols].median())

cat_cols = data.select_dtypes(include=["object"]).columns
data[cat_cols] = data[cat_cols].fillna("missing")

print("Nulls after fill (sample):")
print(data.isna().sum().sort_values(ascending=False).head(10))


Nulls after fill (sample):
TransactionID    0
V231             0
V242             0
V241             0
V240             0
V239             0
V238             0
V237             0
V236             0
V235             0
dtype: int64


In [4]:

# --- Drop unused / high-missing columns (keep only what you need for your engineered feature set)
cols_to_drop = (
    [f"V{i}" for i in range(1, 340)]
    + [f"id_{i:02d}" for i in range(1, 39) if f"id_{i:02d}" not in ["id_31", "id_30", "id_33", "id_36", "id_02"]]
    + [f"D{i}" for i in range(5, 16)]
    + [f"M{i}" for i in range(1, 10)]
    + [f"C{i}" for i in range(1, 15)]
    + ["P_emaildomain", "R_emaildomain"]
)

data = data.drop(columns=cols_to_drop, errors="ignore")
print("Shape after drop:", data.shape)


Shape after drop: (590540, 26)


## Feature engineering (human interpretable)
Reuses your existing `add_features()` logic.

In [5]:
import numpy as np
import pandas as pd
from collections import Counter

def rolling_count_seconds(df: pd.DataFrame, entity_col: str, time_col: str, window_sec: int) -> np.ndarray:
    """
    For each row, count number of rows with the same entity within [time - window_sec, time].
    Returns an array aligned to df.index. Sorts within each group; overall DF row order is preserved.
    """
    counts = np.zeros(len(df), dtype=int)
    for _, group in df.groupby(entity_col):
        g = group.sort_values(time_col)
        times = g[time_col].to_numpy()
        idx = g.index.to_numpy()
        left = 0
        for right in range(len(times)):
            while times[right] - times[left] > window_sec:
                left += 1
            counts[idx[right]] = right - left + 1
    return counts

def add_features(data: pd.DataFrame) -> pd.DataFrame:
    out = data.copy()

    # Time features
    if "TransactionDT" in out.columns:
        day = np.floor(out["TransactionDT"] / (24*60*60)).astype("int64")
        hour = np.floor((out["TransactionDT"] % (24*60*60)) / 3600).astype("int64")
        out["Transaction_day"] = day
        out["Transaction_hour"] = hour
        out["is_night_txn"] = out["Transaction_hour"].isin(range(0, 7)).astype("int8")

        # Per-card/day transaction count
        required = {"card1", "Transaction_day", "TransactionID"}
        if required.issubset(out.columns):
            grp = out.groupby(["card1", "Transaction_day"])["TransactionID"].transform("count")
            out["trans_per_card_day"] = grp.astype("int32")
        else:
            out["trans_per_card_day"] = 1

    # Per-card amount statistics and aliases
    if {"card1", "TransactionAmt"}.issubset(out.columns):
        stats = (
            out.groupby("card1")["TransactionAmt"]
            .agg(avg_amt_per_card="mean", amt_std_per_card="std")
            .reset_index()
        )
        out = out.merge(stats, on="card1", how="left")

        if "avg_amt_per_card" in out.columns:
            out["avg_amt_per_card_y"] = out["avg_amt_per_card"]
        if "amt_std_per_card" in out.columns:
            out["amt_std_per_card_y"] = out["amt_std_per_card"]

        # Fill NaNs in std with its mean
        if "amt_std_per_card_y" in out.columns:
            std_mean = out["amt_std_per_card_y"].mean()
            out["amt_std_per_card_y"] = out["amt_std_per_card_y"].fillna(std_mean)

        # Ratio and deviation features
        if {"TransactionAmt", "avg_amt_per_card_y"}.issubset(out.columns):
            out["freq_ratio_card_amt"] = out["TransactionAmt"] / (out["avg_amt_per_card_y"] + 1e-5)
        if {"TransactionAmt", "avg_amt_per_card_y", "amt_std_per_card_y"}.issubset(out.columns):
            out["amt_deviation_card"] = (
                (out["TransactionAmt"] - out["avg_amt_per_card_y"]).abs()
                / (out["amt_std_per_card_y"] + 1e-5)
            )

    # Rolling counts per card over windows and time since last txn (no global sort)
    if {"card1", "TransactionDT"}.issubset(out.columns):
        out["trans_last_1h_card"]  = rolling_count_seconds(out, "card1", "TransactionDT", 3600).astype("int32")
        out["trans_last_6h_card"]  = rolling_count_seconds(out, "card1", "TransactionDT", 21600).astype("int32")
        out["trans_last_24h_card"] = rolling_count_seconds(out, "card1", "TransactionDT", 86400).astype("int32")
        out["time_since_last_txn_card"] = out.groupby("card1")["TransactionDT"].diff().fillna(0).astype("int64")

    # Unique devices per card in last 24h
    if {"DeviceInfo", "card1", "TransactionDT"}.issubset(out.columns):
        devices_per_card = np.zeros(len(out), dtype=int)
        for _, group in out.groupby("card1"):
            g = group.sort_values("TransactionDT")
            times = g["TransactionDT"].to_numpy()
            devices = g["DeviceInfo"].to_numpy()
            idx = g.index.to_numpy()
            left = 0
            counter = Counter()
            unique_count = 0
            for right in range(len(times)):
                d_r = devices[right]
                if counter[d_r] == 0:
                    unique_count += 1
                counter[d_r] += 1
                while times[right] - times[left] > 86400:
                    d_l = devices[left]
                    counter[d_l] -= 1
                    if counter[d_l] == 0:
                        unique_count -= 1
                        del counter[d_l]
                    left += 1
                devices_per_card[idx[right]] = unique_count
        out["devices_per_card_24h"] = devices_per_card

    # Unique cards per device in last 24h
    if {"DeviceInfo", "card1", "TransactionDT"}.issubset(out.columns):
        cards_per_device = np.zeros(len(out), dtype=int)
        for _, group in out.groupby("DeviceInfo"):
            g = group.sort_values("TransactionDT")
            times = g["TransactionDT"].to_numpy()
            cards = g["card1"].to_numpy()
            idx = g.index.to_numpy()
            left = 0
            counter = Counter()
            unique_count = 0
            for right in range(len(times)):
                c_r = cards[right]
                if counter[c_r] == 0:
                    unique_count += 1
                counter[c_r] += 1
                while times[right] - times[left] > 86400:
                    c_l = cards[left]
                    counter[c_l] -= 1
                    if counter[c_l] == 0:
                        unique_count -= 1
                        del counter[c_l]
                    left += 1
                cards_per_device[idx[right]] = unique_count
        out["cards_per_device_24h"] = cards_per_device

    # Change flags per card based on previous observation of the same card
    # These use group-wise shift without changing global row order.
    if {"card1", "DeviceInfo"}.issubset(out.columns):
        out["device_change_flag"] = (out.groupby("card1")["DeviceInfo"].shift() != out["DeviceInfo"]).astype("int32")
    if {"card1", "id_31"}.issubset(out.columns):
        out["browser_change_flag"] = (out.groupby("card1")["id_31"].shift() != out["id_31"]).astype("int32")
    if {"card1", "id_30"}.issubset(out.columns):
        out["os_change_flag"] = (out.groupby("card1")["id_30"].shift() != out["id_30"]).astype("int32")
    if {"card1", "id_33"}.issubset(out.columns):
        out["screen_resolution_change_flag"] = (out.groupby("card1")["id_33"].shift() != out["id_33"]).astype("int32")
    if {"card1", "id_02"}.issubset(out.columns):
        out["geo_change_flag"] = (out.groupby("card1")["id_02"].shift() != out["id_02"]).astype("int32")

    # Unique counts by transform
    if {"card1", "id_36"}.issubset(out.columns):
        out["unique_ip_per_card"] = out.groupby("card1")["id_36"].transform("nunique")
    if {"card1", "id_36"}.issubset(out.columns):
        out["unique_cards_per_ip"] = out.groupby("id_36")["card1"].transform("nunique")

    # Ratios using earlier features
    if {"trans_per_card_day", "devices_per_card_24h"}.issubset(out.columns):
        out["card_device_use_ratio"] = out["trans_per_card_day"] / (out["devices_per_card_24h"] + 1)
    if {"card1", "time_since_last_txn_card"}.issubset(out.columns):
        mean_time = out.groupby("card1")["time_since_last_txn_card"].transform("mean")
        out["time_diff_ratio_to_card_mean"] = out["time_since_last_txn_card"] / (mean_time + 1e-5)

    # Distance ratio
    if {"dist1", "dist2"}.issubset(out.columns):
        out["dist_ratio"] = out["dist1"] / (out["dist2"].replace(0, np.nan) + 1e-5)

    return out


In [6]:

# --- Sort chronologically (IMPORTANT for time-aware split + time_diff features)
data = data.sort_values("TransactionDT").reset_index(drop=True)

# --- Apply feature engineering on the full chronological stream
# This allows test rows to use historical context from train rows for rolling features.
data_feat = add_features(data)

print("After feature engineering:", data_feat.shape)
print("New columns (sample):", [c for c in data_feat.columns if c not in data.columns][:15])


After feature engineering: (590540, 52)
New columns (sample): ['Transaction_day', 'Transaction_hour', 'is_night_txn', 'trans_per_card_day', 'avg_amt_per_card', 'amt_std_per_card', 'avg_amt_per_card_y', 'amt_std_per_card_y', 'freq_ratio_card_amt', 'amt_deviation_card', 'trans_last_1h_card', 'trans_last_6h_card', 'trans_last_24h_card', 'time_since_last_txn_card', 'devices_per_card_24h']


In [7]:

# --- Build keys table for traceability (NOT used as features)
keys = data_feat[["TransactionID", "TransactionDT", "isFraud"]].copy()


In [8]:

# --- Drop raw columns that you *used* for feature engineering but do not want as model inputs
cols_to_delete = [
    "TransactionID",
    "addr1", "addr2",
    "id_02", "id_30", "id_31", "id_33", "id_36",
    "DeviceType", "DeviceInfo",
    "avg_amt_per_card", "amt_std_per_card",
]

def drop_columns_safe(df: pd.DataFrame, cols):
    return df.drop(columns=[c for c in cols if c in df.columns], errors="ignore")

data_model = drop_columns_safe(data_feat, cols_to_delete)

# Separate X/y, and remove raw TransactionDT from FEATURES (we keep derived time features instead)
y = data_model["isFraud"].astype(int)
X = data_model.drop(columns=["isFraud"], errors="ignore").copy()

if "TransactionDT" in X.columns:
    X = X.drop(columns=["TransactionDT"])  # avoid SMOTE/time artifacts

print("X shape:", X.shape, "y shape:", y.shape)


X shape: (590540, 38) y shape: (590540,)


In [9]:

# --- Define the exact feature set to keep (matches your thesis feature list)
categorical_cols = ["ProductCD", "card4", "card6"]

numeric_cols = [
    "TransactionAmt", "card1", "card2", "card3", "card5", "dist1", "dist2",
    "D1", "D2", "D3", "D4",
    "Transaction_day", "Transaction_hour", "is_night_txn",
    "trans_per_card_day",
    "avg_amt_per_card_y", "amt_std_per_card_y",
    "freq_ratio_card_amt", "amt_deviation_card",
    "trans_last_1h_card", "trans_last_6h_card", "trans_last_24h_card",
    "time_since_last_txn_card",
    "devices_per_card_24h", "cards_per_device_24h",
    "device_change_flag", "browser_change_flag", "os_change_flag",
    "screen_resolution_change_flag", "geo_change_flag",
    "unique_ip_per_card", "unique_cards_per_ip",
    "card_device_use_ratio", "time_diff_ratio_to_card_mean",
    "dist_ratio",
]

feature_cols = categorical_cols + numeric_cols

# Reindex to force consistent column order; fill missing with 0
X = X.reindex(columns=feature_cols, fill_value=0)

# Basic checks
missing_feats = [c for c in feature_cols if c not in X.columns]
print("Missing feature cols after reindex (should be 0):", len(missing_feats))
print("Train fraud rate (overall):", y.mean())


Missing feature cols after reindex (should be 0): 0
Train fraud rate (overall): 0.03499000914417313


In [10]:

# --- Chronological Train/Test split (real transactions only)
# Use 80/20 by time. (Change TRAIN_FRAC if needed.)
TRAIN_FRAC = 0.80
n = len(X)
split_idx = int(n * TRAIN_FRAC)

X_train_lgb = X.iloc[:split_idx].reset_index(drop=True)
X_test_lgb  = X.iloc[split_idx:].reset_index(drop=True)

y_train = y.iloc[:split_idx].reset_index(drop=True)
y_test  = y.iloc[split_idx:].reset_index(drop=True)

train_keys = keys.iloc[:split_idx].reset_index(drop=True)
test_keys  = keys.iloc[split_idx:].reset_index(drop=True)

# Add row_id (per-split)
train_keys.insert(0, "row_id", np.arange(len(train_keys)))
test_keys.insert(0, "row_id", np.arange(len(test_keys)))

print("Train:", X_train_lgb.shape, "Test:", X_test_lgb.shape)
print("Train fraud%:", y_train.mean()*100, "Test fraud%:", y_test.mean()*100)

# Ensure chronological split boundary
assert train_keys["TransactionDT"].max() <= test_keys["TransactionDT"].min(),     "Time split violated: train has later timestamps than test."


Train: (472432, 38) Test: (118108, 38)
Train fraud%: 3.5135215226741625 Test fraud%: 3.4409184813899145


In [11]:

# --- Save LightGBM raw engineered features + labels + keys
X_train_path = os.path.join(datapath, "X_train_lgb.csv")
X_test_path  = os.path.join(datapath, "X_test_lgb.csv")
y_train_path = os.path.join(datapath, "y_train.csv")
y_test_path  = os.path.join(datapath, "y_test.csv")
train_keys_path = os.path.join(datapath, "train_keys.csv")
test_keys_path  = os.path.join(datapath, "test_keys.csv")

X_train_lgb.to_csv(X_train_path, index=False)
X_test_lgb.to_csv(X_test_path, index=False)

y_train.to_frame("isFraud").to_csv(y_train_path, index=False)
y_test.to_frame("isFraud").to_csv(y_test_path, index=False)

train_keys.to_csv(train_keys_path, index=False)
test_keys.to_csv(test_keys_path, index=False)

print("Saved:")
print(" -", X_train_path)
print(" -", X_test_path)
print(" -", y_train_path)
print(" -", y_test_path)
print(" -", train_keys_path)
print(" -", test_keys_path)


Saved:
 - /content/drive/MyDrive/RThesis/X_train_lgb.csv
 - /content/drive/MyDrive/RThesis/X_test_lgb.csv
 - /content/drive/MyDrive/RThesis/y_train.csv
 - /content/drive/MyDrive/RThesis/y_test.csv
 - /content/drive/MyDrive/RThesis/train_keys.csv
 - /content/drive/MyDrive/RThesis/test_keys.csv


In [12]:

# --- Build scaled features for unsupervised models (OHE + StandardScaler)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

def make_ohe():
    # sklearn <1.2 uses sparse=..., >=1.2 uses sparse_output=...
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

ohe = make_ohe()

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", ohe, categorical_cols),
        ("num", "passthrough", numeric_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=True,
)

pipe_scaled = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("scale", StandardScaler())
])

X_train_scaled = pipe_scaled.fit_transform(X_train_lgb)
X_test_scaled  = pipe_scaled.transform(X_test_lgb)

# Feature names
feature_names = pipe_scaled.named_steps["preprocess"].get_feature_names_out()
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled_df  = pd.DataFrame(X_test_scaled, columns=feature_names)

print("Scaled shapes:", X_train_scaled_df.shape, X_test_scaled_df.shape)

# Save
X_train_scaled_path = os.path.join(datapath, "X_train_scaled.csv")
X_test_scaled_path  = os.path.join(datapath, "X_test_scaled.csv")
pipe_path = os.path.join(datapath, "pipe_scaled.joblib")

X_train_scaled_df.to_csv(X_train_scaled_path, index=False)
X_test_scaled_df.to_csv(X_test_scaled_path, index=False)
joblib.dump(pipe_scaled, pipe_path)

print("Saved scaled data + pipeline:")
print(" -", X_train_scaled_path)
print(" -", X_test_scaled_path)
print(" -", pipe_path)


Scaled shapes: (472432, 50) (118108, 50)
Saved scaled data + pipeline:
 - /content/drive/MyDrive/RThesis/X_train_scaled.csv
 - /content/drive/MyDrive/RThesis/X_test_scaled.csv
 - /content/drive/MyDrive/RThesis/pipe_scaled.joblib
